In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import narsil
from narsil.segmentation.run import loadNet, segmentPosDirectory
from torchvision import transforms
import matplotlib.pyplot as plt
from skimage.io import imread
import torch
from narsil.liverun.utils import resizeOneImage, tensorizeOneImage
import numpy as np
from scipy.signal import find_peaks

In [3]:
rootDir = '/mnt/sda2/Praneeth/ASTFISH/'
models = {
    'cells' : rootDir + 'ASTPaper/segModels/mixed10epochs_betterscale_contrastAdjusted.pth',
    'channels' : rootDir + 'ASTPaper/segModels/channels.pth'
}
device = "cuda:1"

In [4]:
cellSegNet = loadNet(models['cells'], device)

In [5]:
channelSegNet = loadNet(models['channels'], device)

In [6]:
def runNet(net, imgFileName):
    img = imread(imgFileName)
    img = (img - np.mean(img))/np.std(img)
    imgTransformations = transforms.Compose([resizeOneImage((1504, 4096), (1504, 4096)),
                                 tensorizeOneImage(2)])
    output = net(imgTransformations(img).to(device))
    output = torch.sigmoid(output)
    out_numpy = output.cpu().detach().numpy().squeeze(0).squeeze(0)
    
    return out_numpy

In [7]:
for position in range(122, 130):
    imgFileName = '/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos' + str(position) + '/phaseFast/img_000000000.tiff'
    channelSegMask = runNet(channelSegNet, imgFileName)
    plt.figure()
    plt.imshow(channelSegMask)
    plt.show()

In [9]:
plt.imshow(channelSegMask)

In [10]:
np.sum(channelSegMask, axis = 1)

array([41.346973 ,  5.495246 ,  5.6024246, ...,  7.6095085, 10.837442 ,
       68.894325 ], dtype=float32)